In [ ]:
#| hide

from fh_saas.utils_webhook import *
import os
import hmac
import hashlib

## Test: Signature Verification

In [ ]:
#| hide

secret = "test_secret_key"
payload = '{"event": "test", "data": "value"}'

# Generate valid signature
expected_sig = hmac.new(
    secret.encode('utf-8'),
    payload.encode('utf-8'),
    hashlib.sha256
).hexdigest()

# Test with plain hex
assert verify_webhook_signature(payload, expected_sig, secret) == True
print("✓ Valid signature verified (plain hex)")

# Test with "sha256=" prefix
assert verify_webhook_signature(payload, f"sha256={expected_sig}", secret) == True
print("✓ Valid signature verified (with prefix)")

✓ Valid signature verified (plain hex)
✓ Valid signature verified (with prefix)


In [ ]:
#| hide

invalid_sig = "0" * 64  # Wrong signature
assert verify_webhook_signature(payload, invalid_sig, secret) == False
print("✓ Invalid signature rejected")

✓ Invalid signature rejected


In [ ]:
#| hide

modified_payload = '{"event": "test", "data": "modified"}'
assert verify_webhook_signature(modified_payload, expected_sig, secret) == False
print("✓ Modified payload rejected")

✓ Modified payload rejected


In [ ]:
#| hide

os.environ.pop('WEBHOOK_SECRET', None)  # Ensure env var not set

try:
    verify_webhook_signature(payload, expected_sig)
    assert False, "Should have raised ValueError"
except ValueError as e:
    assert "WEBHOOK_SECRET" in str(e)
    print("✓ Missing secret raises error")

✓ Missing secret raises error


## Test: Idempotency Checking

In [ ]:
#| hide

class MockDB:
    def __init__(self):
        self.events = []
    
    def q(self, query, params):
        key = params[0]
        return [e for e in self.events if e.get('idempotency_key') == key]

mock_db = MockDB()

# First check - should return False (not duplicate)
assert check_idempotency(mock_db, 'unique_key_1') == False
print("✓ New event not marked as duplicate")

# Add event
mock_db.events.append({'webhook_id': 'wh_123', 'idempotency_key': 'unique_key_1'})

# Second check - should return True (duplicate)
assert check_idempotency(mock_db, 'unique_key_1') == True
print("✓ Duplicate event detected")

✓ New event not marked as duplicate
✓ Duplicate event detected


## Test: Webhook Event Logging

In [ ]:
#| hide

class MockDBWithInsert:
    def __init__(self):
        self.inserted = []
    
    def insert(self, record, table):
        record['_table'] = table
        self.inserted.append(record)

mock_db = MockDBWithInsert()

log_webhook_event(
    db=mock_db,
    webhook_id='wh_test_123',
    source='test_source',
    event_type='test.event',
    payload={'key': 'value'},
    signature='abc123',
    idempotency_key='idem_123',
    status='pending'
)

assert len(mock_db.inserted) == 1
record = mock_db.inserted[0]
assert record['webhook_id'] == 'wh_test_123'
assert record['source'] == 'test_source'
assert record['event_type'] == 'test.event'
assert record['idempotency_key'] == 'idem_123'
assert record['status'] == 'pending'
assert record['_table'] == 'webhook_events'
print("✓ Webhook event logged correctly")

✓ Webhook event logged correctly


## Test: Update Webhook Status

In [ ]:
#| hide

class MockDBWithUpdate:
    def __init__(self):
        self.updates = []
    
    def update(self, data, table, key_col, key_val):
        self.updates.append({
            'data': data,
            'table': table,
            'key_col': key_col,
            'key_val': key_val
        })

mock_db = MockDBWithUpdate()

# Update without error
update_webhook_status(mock_db, 'wh_123', 'completed')
assert len(mock_db.updates) == 1
assert mock_db.updates[0]['data']['status'] == 'completed'
assert 'processed_at' in mock_db.updates[0]['data']
print("✓ Status updated successfully")

# Update with error message
update_webhook_status(mock_db, 'wh_124', 'failed', 'Test error')
assert len(mock_db.updates) == 2
assert mock_db.updates[1]['data']['status'] == 'failed'
assert mock_db.updates[1]['data']['error_message'] == 'Test error'
print("✓ Status updated with error message")

✓ Status updated successfully
✓ Status updated with error message


## Test: Process Webhook Integration

In [ ]:
import asyncio

# Mock comprehensive DB
class MockDBComplete:
    def __init__(self):
        self.events = []
        self.inserted = []
        self.updates = []
    
    def q(self, query, params):
        key = params[0]
        return [e for e in self.events if e.get('idempotency_key') == key]
    
    def insert(self, record, table):
        self.inserted.append(record)
        self.events.append(record)
    
    def update(self, data, table, key_col, key_val):
        self.updates.append({'data': data, 'key_val': key_val})

# Test handler
async def test_handler(payload, db):
    return {'processed': True, 'count': len(payload)}

# Generate valid signature
secret = "test_secret"
raw_body = '{"test": "data"}'
signature = hmac.new(
    secret.encode('utf-8'),
    raw_body.encode('utf-8'),
    hashlib.sha256
).hexdigest()

mock_db = MockDBComplete()

# Process webhook
result = await process_webhook(
    db=mock_db,
    webhook_id='wh_integration_test',
    source='test',
    event_type='test.event',
    payload={'test': 'data'},
    signature=signature,
    idempotency_key='idem_integration',
    raw_body=raw_body,
    handler=test_handler,
    secret=secret
)

assert result['status'] == 'success'
assert result['result']['processed'] == True
assert len(mock_db.inserted) == 1  # Event logged
assert len(mock_db.updates) == 1  # Status updated
print("✓ Full webhook processing flow works")

✓ Full webhook processing flow works


In [ ]:
#| hide

result2 = await process_webhook(
    db=mock_db,
    webhook_id='wh_integration_test_2',
    source='test',
    event_type='test.event',
    payload={'test': 'data'},
    signature=signature,
    idempotency_key='idem_integration',  # Same key
    raw_body=raw_body,
    handler=test_handler,
    secret=secret
)

assert result2['status'] == 'duplicate'
print("✓ Duplicate webhook detected and rejected")

✓ Duplicate webhook detected and rejected


In [ ]:
#| hide
result3 = await process_webhook(
    db=mock_db,
    webhook_id='wh_invalid',
    source='test',
    event_type='test.event',
    payload={'test': 'data'},
    signature='invalid_signature',
    idempotency_key='idem_new',
    raw_body=raw_body,
    handler=test_handler,
    secret=secret
)

assert result3['status'] == 'error'
assert 'Invalid signature' in result3['message']
print("✓ Invalid signature rejected")

✓ Invalid signature rejected
